# Lab: Even More Linear Regression
## CMSE 381 - Spring 2023
## Lecture 7, Jan 25, 2023

In the last few lectures, we have focused on linear regression, that is, fitting models of the form 
$$
Y =  \beta_0 +  \beta_1 X_1 +  \beta_2 X_2 + \cdots +  \beta_pX_p + \varepsilon
$$
In this lab, we will continue to use two different tools for linear regression. 
- [Scikit learn](https://scikit-learn.org/stable/index.html) is arguably the most used tool for machine learning in python 
- [Statsmodels](https://www.statsmodels.org) provides many of the statisitcial tests we've been learning in class

This lab will cover two ideas: 
- Categorical variables and how to represent them as dummy variables. 
- How to build interaction terms and pass them into your favorite model.

In [1]:
# As always, we start with our favorite standard imports. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import statsmodels.formula.api as smf

# Playing with multi-level variables 

## The wrong way

Ok, we're going to do this incorrectly to start. You pull in the `Auto` data set. You were so proud of yourself for remembering to fix the problems with the `horsepower` column that you conveniently forgot that the column with information about country of origin (`origin`) has a bunch of integers in it, representing:
- 1: `American`
- 2: `European`
- 3: `Japanese`.

In [2]:
Auto_df = pd.read_csv('Auto.csv')
Auto_df = Auto_df.replace('?', np.nan)
Auto_df = Auto_df.dropna()
Auto_df.horsepower = Auto_df.horsepower.astype('int')


Auto_df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'name'],
      dtype='object')

You then go on your merry way building the model 
$$
\texttt{mpg} = \beta_0 + \beta_1 \cdot \texttt{origin}. 
$$

In [3]:
from sklearn.linear_model import LinearRegression

X = Auto_df.origin.values
X = X.reshape(-1, 1)
y = Auto_df.mpg.values

regr = LinearRegression()

regr.fit(X,y)

print('beta_1 = ', regr.coef_[0])
print('beta_0 = ', regr.intercept_)

beta_1 =  5.4765474801914475
beta_0 =  14.811973615412462


&#9989; **<font color=red>Q:</font>** What does your model predict for each of the three types of cars? 

In [ ]:
# Your code here

&#9989; **<font color=red>Q:</font>** Is it possible for your model to predict that both American and Japanese cars have `mpg` below European cars? 

Your answer here.

## The right way

Ok, so you figure out your problem and decide to load in your data and fix the `origin` column to have names as entries.

In [4]:
convertOrigin= {1: 'American', 2:'European', 3:'Japanese'}

# This command swaps out each number n for convertOrigin[n], making it one of
# the three strings instead of an integer now.
Auto_df.origin = Auto_df.origin.apply(lambda n: convertOrigin[n])
Auto_df

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,American,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,American,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,American,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,American,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,American,ford torino
...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86,2790,15.6,82,American,ford mustang gl
393,44.0,4,97.0,52,2130,24.6,82,European,vw pickup
394,32.0,4,135.0,84,2295,11.6,82,American,dodge rampage
395,28.0,4,120.0,79,2625,18.6,82,American,ford ranger


Below is a quick code that automatically generates our dummy variables. Yay for not having to code that mess ourselves!

In [9]:
origin_dummies_df = pd.get_dummies(Auto_df.origin, prefix = 'origin')
origin_dummies_df.iloc[1:30, ]

,origin_American,origin_European,origin_Japanese
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0
10,1,0,0


&#9989; **<font color=red>Q:</font>** What is the interpretation of each column in the `origin_dummies_df` data frame?

*Your answer here*

I pass these new dummy variables into my `scikit-learn` linear regression model and get the following coefficients

In [19]:
origin_dummies_df.iloc[:, 0:2]

,origin_American,origin_European
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
392,1,0
393,0,1
394,1,0
395,1,0


In [21]:
X = origin_dummies_df.iloc[:, 0:2].values
y = Auto_df.mpg

regr = LinearRegression()

regr.fit(X,y)

print('Coefs = ', regr.coef_)
print('Intercept = ', regr.intercept_)

Coefs =  [-10.41716352  -2.84769173]
Intercept =  30.4506329113924


&#9989; **<font color=red>Q:</font>** Now what does your model predict for each of the three types of cars? 

In [ ]:
# Your code here

## Another right way

Ok, fine, I'll cave, I made you do it the hard way but you got to see how the innards worked, so maybe it's not all bad ;) 

The following code does the same thing as above, but because `statsmodels` has built in tools to handle categorical variables in a data frame, it does the hard work for you. 

In [10]:
est = smf.ols('mpg ~ origin', Auto_df).fit()
est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,20.0335,0.409,49.025,0.000,19.230,20.837
origin[T.European],7.5695,0.877,8.634,0.000,5.846,9.293
origin[T.Japanese],10.4172,0.828,12.588,0.000,8.790,12.044


&#9989; **<font color=red>Q:</font>** What is the model learned from the above printout? Be specific in terms of your dummy variables. 

Your answer here

![Stop Icon](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Vienna_Convention_road_sign_B2a.svg/180px-Vienna_Convention_road_sign_B2a.svg.png)

Great, you got to here! Hang out for a bit, there's more lecture before we go on to the next portion. 

# Interaction Terms 

Below is the code I have for generating the tables shown on the slides. 

In [22]:
Advertising_df = pd.read_csv('Advertising.csv', index_col = 0)
Advertising_df.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [23]:
Advertising_df['TVxRadio'] = Advertising_df.TV * Advertising_df.Radio
Advertising_df.head()

,TV,Radio,Newspaper,Sales,TVxRadio
1,230.1,37.8,69.2,22.1,8697.78
2,44.5,39.3,45.1,10.4,1748.85
3,17.2,45.9,69.3,9.3,789.48
4,151.5,41.3,58.5,18.5,6256.95
5,180.8,10.8,58.4,12.9,1952.64


In [24]:
est = smf.ols('Sales ~ TV + Radio + TVxRadio', Advertising_df).fit()
est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,6.7502,0.248,27.233,0.000,6.261,7.239
TV,0.0191,0.002,12.699,0.000,0.016,0.022
Radio,0.0289,0.009,3.241,0.001,0.011,0.046
TVxRadio,0.0011,5.24e-05,20.727,0.000,0.001,0.001


&#9989; **<font color=red>Do this:</font>** Using the Auto data set, train the model 
$$
\texttt{mpg} = \beta_0 + \beta_1\cdot \texttt{weight} + \beta_2\cdot \texttt{horsepower} + \beta_3\cdot \texttt{weight x horsepower}.
$$
Is the interaction term adding value to the model? 

In [ ]:
# Your code here



-----
### Congratulations, we're done!


<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.